Congratulations, you have just landed your first job as a data analyst at Nearly New
Nautical! Nearly New Nautical is a website that allows users to advertise their used
boats for sale. The marketing team is preparing a weekly newsletter for boat owners.
The newsletter is designed to help sellers to get more views of their boat, as well as
stay on top of market trends. The Head of Marketing has laid out an ambitious goal
of increasing the number of readers by 75% this year.

They would like you to take a look at the recent data and help them learn more
about the characteristics of the most viewed boat listings in the last 7 days - is it the
most expensive boats that get the most views? Are there common features among
the most viewed boats?

Your presentation should be no longer than ten minutes. It should include no more
than eight slides of content. You will be presenting to the Head of Marketing.

# Notes and Plans

# Cleaning and Wrangling Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('boat_data.csv')

price = df['Price'].values
df['currency'] = [s.split(' ')[0] for s in price]
df['amount'] = [float(s.split(' ')[1]) for s in price]
    
POUND2EUR = 1.16  # pound sterling to euro
DKK2EUR = 0.13  # danish krone to euro
CHF2EUR = 0.99  # swiss franc to euro

df['Price_Euros'] = np.nan 
for i, cur in enumerate(df['currency']):
    if cur == 'CHF':
        df['Price_Euros'][i] = df['amount'][i]*CHF2EUR
    elif cur == 'DKK':
        df['Price_Euros'][i] = df['amount'][i]*DKK2EUR
    elif cur == 'EUR':
        df['Price_Euros'][i] = df['amount'][i]
    else:
        df['Price_Euros'][i] = df['amount'][i]*POUND2EUR

# Extract Country values out of Location column
df['Country'] = ""
for i, l in enumerate(df['Location']):
    if not df['Location'].isna()[i]:  # skip NaN values in Location columns
        country = l.split('Â»')[0]
        # print(country)
        df['Country'][i] = country

# Eliminate 'power boats' from text in Manufacturer column
for i, m in enumerate(df['Manufacturer']):
    if not df['Manufacturer'].isna()[i]:   # filter for non-empty values 
        df['Manufacturer'][i] = m.replace('power boats', "")   # remove characters 'power boats' for each row


df['Size'] = ""
# skip NaN values in Location columns
for i in range(len(df)):
    if not df['Length'].isna()[i] and not df['Width'].isna()[i]:
        df['Size'][i] = df['Length'][i]*df['Width'][i]
    else: 
        df['Size'][i] = np.NaN

# replace 0 with nan
df['Year Built'].replace(0, np.nan, inplace=True)  

# Create new column 'Condition' that records used or new
df['Condition'] = ''
for i, t in enumerate(df['Type']):
    if not df['Type'].isna()[i]:
        df['Condition'][i] = str(t).split(' ')[0]

reordered_columns = ['Number of views last 7 days', 'Price', 'currency', 'amount', 'Price_Euros', 'Boat Type', 
                     'Manufacturer', 'Type', 'Condition', 'Year Built', 'Length', 'Width', 'Size',
                     'Material', 'Location', 'Country' ]
df = df[reordered_columns]

df.sort_values(by=['Number of views last 7 days'], ascending = False, inplace=True)

df.to_csv('new_boat_data.csv')


KeyboardInterrupt



In [ ]:
# There are duplicate country categories because some have space in the end
### next time use lstrip or rstrip
for i, country in enumerate(df['Country']):
    # print(country)
    try:
        if country[-1] == ' ':
            df['Country'][i] = country[:-1]
    except:
        pass

In [ ]:
df = pd.read_csv('useful_boat_data.csv')

In [ ]:
top_countries_views = df.groupby('Country')['Number of views last 7 days'].sum().sort_values(ascending=False)
top20country = top_countries_views.index[:20].values

def create_palette_dict(keys, palette='Paired'):
    color_values = sns.color_palette(palette, n_colors = len(keys))
    random.shuffle(keys)
    return {keys[i]: color_values[i] for i in range(len(keys))}

palette_dict = create_palette_dict(keys = top20country, palette='Paired')

In [ ]:
def top_barplot(plot_data, n=10, xlabel=' ', ylabel=' ', title=' ', color_palette = None):
    
    fig, ax = plt.subplots(figsize = (10,6))
    sns.barplot(y=plot_data.index, x = plot_data.values, palette = palette_dict, ax=ax)
    ax.set(
        xlabel = xlabel, 
        ylabel = ylabel, 
        title = title
    )
    plt.show()

In [ ]:
n=10
plot_data = df.groupby('Country')['Number of views last 7 days'].sum().sort_values(ascending=False)[:n]

fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(y=plot_data.index, x = plot_data.values, palette = palette_dict, ax=ax)
ax.set(
    xlabel = 'Total Number of Views Last 7 Days', 
    ylabel = 'Country', 
    title = 'Top 10 Countries in Total Views'
)
plt.show()

In [ ]:
plot_data = df['Country'].value_counts()[:15]

fig, ax = plt.subplots(figsize = (10,6))

sns.barplot(y=plot_data.index, x = plot_data.values,palette=palette_dict,  ax=ax)
ax.set(xlabel = 'Number of listings', ylabel = 'Country', title = 'Total Number of Listings Based on Country')
plt.show()